<a href="https://colab.research.google.com/github/HasnatHridoy/BD-antibiotic-drug-market-analysis/blob/main/Antibiotc_price_scraping_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-colab-selenium -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.6 MB/s eta 0:00:00


In [ ]:
#this driver is specifically used for google colab
#please inintialize your driver properly if you want to run it locally
import google_colab_selenium as gs
driver = gs.Chrome()



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [ ]:
# -------------- scraping part -----------------

In [ ]:
#link scraper function

def link_scraper(links):

  if isinstance(links, str):
    links = [links]

  linking = []

  for link in links:

    driver.get(link)
    print(f'working on {driver.current_url}')

    try:

      div_elements = wait.until(
          EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".col-xs-12.col-sm-6 a"))
      )

      # Extract href
      for element in div_elements:
          linking.append(element.get_attribute("href"))

    except Exception as e:
      print(f"An error occurred while scraping links: {e}")

  return linking

In [ ]:
wait = WebDriverWait(driver, 10)

In [ ]:
try:

  driver.get("https://medex.com.bd/drug-classes")
  print(f"Successfully navigated to: {driver.current_url}")

except Exception as e:
    print(f"An error occurred: {e}")

Successfully navigated to: https://medex.com.bd/drug-classes


In [ ]:
try:
    # click antimicrobial button
    antibio_button = wait.until(
        EC.element_to_be_clickable((By.LINK_TEXT, "Antimicrobial drugs"))
    )

    antibio_button.click()

    url = driver.current_url
    print(f"Successfully navigated to: {driver.current_url}")


except Exception as e:
    print(f"An error occurred: {e}")

Successfully navigated to: https://medex.com.bd/drug-classes/3/antimicrobial-drugs


In [ ]:
links= link_scraper(url)

working on https://medex.com.bd/drug-classes/3/antimicrobial-drugs


In [ ]:
links_2 = link_scraper(links[0])

working on https://medex.com.bd/drug-classes/50/drugs-for-bacterial-infections-anti-bacterial


In [ ]:
links_interm = link_scraper(links_2)

working on https://medex.com.bd/drug-classes/55/4-quinolone-preparations
working on https://medex.com.bd/drug-classes/56/aminoglycosides
working on https://medex.com.bd/drug-classes/57/anti-leprotic-drugs
working on https://medex.com.bd/drug-classes/58/anti-tubercular-antibiotics
working on https://medex.com.bd/drug-classes/59/anti-tubercular-chemotherapeutics
working on https://medex.com.bd/drug-classes/60/benzylpenicillin-phenoxymethyl-penicillin
working on https://medex.com.bd/drug-classes/61/broad-spectrum-penicillins
working on https://medex.com.bd/drug-classes/62/combined-anti-tubercular-preparations
working on https://medex.com.bd/drug-classes/566/ear-anti-infectives-antiseptics
working on https://medex.com.bd/drug-classes/565/eye-anti-infectives-antiseptics
working on https://medex.com.bd/drug-classes/675/fifth-generation-cephalosporins
working on https://medex.com.bd/drug-classes/63/first-generation-cephalosporins
working on https://medex.com.bd/drug-classes/64/fourth-generati

In [ ]:
#two links have sublinks, they need to extract separately
links_workable = links_interm.copy()
link_pop = []

for link in links_interm:

  if 'other-antibiotic' in link or 'probiotic' in link:
    link_pop.append(link)
    links_workable.remove(link)


In [ ]:
links_pop = link_scraper(link_pop)

working on https://medex.com.bd/drug-classes/619/other-antibiotic
working on https://medex.com.bd/drug-classes/657/probiotic


In [ ]:
links_workable.extend(links_pop)

In [ ]:
#some links have duplicate
from collections import Counter

item_counts = Counter(links_workable) #count

duplicates = [item for item, count in item_counts.items() if count > 1]

if duplicates:
  print("duplicates:")
  for duplicate in duplicates:
    print(duplicate)
else:
  print("Nill")

duplicates:
https://medex.com.bd/generics/890/piperacillin-tazobactam
https://medex.com.bd/generics/237/chloramphenicol-ophthalmic
https://medex.com.bd/generics/851/oxytetracycline


In [ ]:
links_workable = list(set(links_workable))

In [ ]:
len(links_workable)

99

In [ ]:
data_extract =[]

In [ ]:
try:

  for link in links_workable:

    parts = link.split('/')
    generic_name = parts[-1]

    driver.get(link+'/brand-names')
    print(f'working on {driver.current_url}')



    table = driver.find_element(By.CLASS_NAME, 'table.gg-table.bindex-table')
    rows = table.find_elements(By.TAG_NAME, 'tr')

    for row in rows:
      cells = row.find_elements(By.TAG_NAME, 'td')

      if not cells:
          continue

      brand_name = cells[0].text
      dosage_form = cells[1].text
      strength = cells[2].text
      company = cells[3].text
      price_pack = cells[4].text

      data_extract.append([generic_name, brand_name, dosage_form, strength, company, price_pack])

except Exception as e:
    print(f"An error occurred while scraping links: {e}")




working on https://medex.com.bd/generics/231/cephalexin/brand-names
working on https://medex.com.bd/generics/1876/probiotic-combination-14-strains/brand-names
working on https://medex.com.bd/generics/110/benzathine-penicillin/brand-names
working on https://medex.com.bd/generics/2069/probiotic-combination-23-billion/brand-names
working on https://medex.com.bd/generics/836/ofloxacin/brand-names
working on https://medex.com.bd/generics/1061/thalidomide/brand-names
working on https://medex.com.bd/generics/209/cefaclor-monohydrate/brand-names
working on https://medex.com.bd/generics/388/doripenem/brand-names
working on https://medex.com.bd/generics/255/imipenem-cilastatin/brand-names
working on https://medex.com.bd/generics/439/rifampicin-isoniazid-pyrazinamide-ethambutol/brand-names
working on https://medex.com.bd/generics/2456/probiotic-combination-bi-07-10-billion/brand-names
working on https://medex.com.bd/generics/225/ceftriaxone-sodium/brand-names
working on https://medex.com.bd/gener

In [ ]:
#--------------------- Data preprocessing part -----------------------

In [ ]:
import re

In [ ]:
#striper function

def extract_price(text):
    #price_unit
    match = re.search(r'৳\s*([\d,.]+)', text)
    if match:
        return float(match.group(1).replace(',', ''))
    return None

def extract_strip_price(text):
    #strip price
    match = re.search(r'\([^)]*৳\s*([\d,.]+)\)', text)
    if match:
        return float(match.group(1).replace(',', ''))
    return None

def extract_pack_size(text):
    #pack

    # Case 1: "A x B" format
    match = re.search(r'\((\d+\s*x\s*\d+)', text)
    if match:
        return match.group(1).strip() + " pcs"

    # Case 2: numeric + unit format
    match = re.search(r"(\d+\s*[a-zA-Z/%']+)", text, re.IGNORECASE)
    if match:
        return match.group(1).strip()

    return None

def extract_pack_type(text):
    #pck_type

    match = re.search(r'\b(bottle|tube|sachet|vial|ampoule|refill|pack|drop)\b', text, re.IGNORECASE)
    if match:
        return match.group(1).lower()

    # If multiplicative pattern exists, assume strip
    if re.search(r'\d+\s*x\s*\d+', text):
        return "strip"

In [ ]:
# preprocessing main function
def process_data(data):
    processed = []
    for row in data:
        pack_price_text = row[5]

        price = extract_price(pack_price_text)
        strip_price = extract_strip_price(pack_price_text)
        pack_size = extract_pack_size(pack_price_text)
        pack_type = extract_pack_type(pack_price_text)

        processed.append(row + [price, strip_price, pack_size, pack_type])
    return processed


In [ ]:
un_list = data_extract.copy()

In [ ]:
pro_list = []

In [ ]:
for item in un_list:
  regular_info = item[:5]
  pack_price_str = item[5]

  if '\n' in pack_price_str:
    pp_entry = pack_price_str.split('\n')

    for entry in pp_entry:
      pro_list.append(regular_info + [entry])

  else:
    pro_list.append(item)

In [ ]:
final_data_2 = process_data(pro_list)

In [ ]:
column_names = ['generics', 'product_name', 'dosage_form', 'strength', 'company', 'ref_p&p', 'unit_price','strip_price','pack_size','pack_type']
df_2 = pd.DataFrame(final_data_2, columns=column_names)

In [ ]:
df_2 = df_2.drop('ref_p&p', axis=1)

In [ ]:
df_2.to_csv('extracted_data.csv', index=False)